# Read Various Data Types from Database

Again, make sure you connect to the isaac database

In [1]:
from pyArango.connection import *

# Connect to the database
conn = Connection(arangoURL="http://iui_arangodb:8529", username="root", password="isaac")

# Open the isaac database / create it if it does not exist
if not conn.hasDatabase("isaac"):
    print("There is no isaac database, did you load it?")
else:
    db = conn["isaac"]
    print("Connected to database")

Connected to database


Check which collections are available to read from:

Read 1D Data
================

To read signal data from the database you can filter by timestamp:

In [14]:
import matplotlib.pyplot as plt


# Edit timestamps timestamp
start_time = 0
end_time = 99999999999
ros_topic = "/gnc/ekf"

aql = ""
if start_time is not None and end_time is not None:
    ros_topic = ros_topic.replace("/", "_")[1:]

    aql = "FOR doc IN " + ros_topic + "\n"\
        + "\tFILTER doc.header.stamp.secs >= " + str(start_time) + \
              " AND doc.header.stamp.secs <= " + str(end_time) + "\n" \
        + "\tRETURN doc";

result = list(db.AQLQuery(aql, rawResults = True))


In [ ]:


# Plot the robot's position during the time interval
time = [msg.get("header").get("stamp").get("secs") - result[0].get("header").get("stamp").get("secs") for msg in result] 
pos_x = [msg.get("pose").get("position").get("x") for msg in result]
pos_y = [msg.get("pose").get("position").get("y") for msg in result]
pos_z = [msg.get("pose").get("position").get("z") for msg in result]

# plt.plot(time,pos_x)
plt.plot(pos_y[:30000])
# plt.plot(time,pos_z)

Read 2D Data
===========
To read data like images:

In [36]:


# Edit timestamps timestamp
start_time = 1616777590
end_time = 1616777600
ros_topic = "/mgt/img_sampler/nav_cam/image_record"


aql = ""
if start_time is not None and end_time is not None:
    ros_topic = ros_topic.replace("/", "_")[1:]

    aql = "FOR doc IN " + ros_topic + "\n"\
        + "\tFILTER doc.header.stamp.secs >= " + str(start_time) + \
              " AND doc.header.stamp.secs <= " + str(end_time) + "\n" \
        + "\tRETURN doc";

result = db.AQLQuery(aql, rawResults = True)


In [ ]:
import cv2
import numpy as np
from cv_bridge import CvBridge, CvBridgeError
print(result[1].keys())
print(len(result[1].get("data")))
array_1 = np.array(result[1].get("data"))
print(len(array_1))
array = array_1.reshape(1,result[1].get("height"), -1)

print(type(array))
print(array.shape)


img = cv2.imdecode(array, cv2.IMREAD_UNCHANGED)
# plt.imshow(img,cmap='gray')
# plt.show()
cv2.imshow('graycsale image',img)